# Notebook initialization

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

In [2]:
def load_features(directory):
    au_features = pd.read_csv('{}/{}/audio_features.csv'.format('../data/output/features',directory), index_col=0)
    im_features = pd.read_csv('{}/{}/image_features.csv'.format('../data/output/features',directory), index_col=0)
    
    # Drop redundant columns
    im_features = im_features.drop(['label'], axis=1)

    # Merge audio and image features
    features = pd.concat([au_features, im_features], axis=1)

    # Only look at clips less than 300s long
    features = features[features.length < 300]
    
    return features

# Labeled audios

## Import data

In [3]:
features = load_features('train')
features.head()

,audio_file,image_file,length,label,percent_silence,ring_count,last_ring_to_end,white_proportion
index,,,,,,,,
0,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,52.740,0,0.342474,0,52.704000,0.831465
1,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,27.000,1,0.958180,0,26.928000,0.977077
2,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,56.088,0,0.208273,0,56.016000,0.853612
3,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,215.640,0,0.122387,0,215.568000,0.823863
4,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,117.216,0,0.407660,2,107.154625,0.880819


## Data Preparation

## Clean Data

In [4]:
print(len(features[features.isnull().any(axis=1)]))
features[features.isnull().any(axis=1)].head()

1


,audio_file,image_file,length,label,percent_silence,ring_count,last_ring_to_end,white_proportion
index,,,,,,,,
14274,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,84.24,1,0.003811,0,84.168,NaN


In [5]:
# just drop the remaning rows with nan values
features = features.dropna()

In [6]:
# Split training and test set
from sklearn import cross_validation
columns = ['label', 'length', 'last_ring_to_end', 'percent_silence', 'ring_count', 'white_proportion']
train, test = cross_validation.train_test_split(features[columns], train_size=0.7, random_state=1000)
y_train = train['label']
X_train = train.drop('label', axis=1)
y_test = test['label']
X_test = test.drop('label', axis=1)

In [7]:
# Scale features
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

## Fit Models

### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
list(zip(columns[1:], lr.coef_[0]))

[('length', -3.814302896584862),
 ('last_ring_to_end', 0.0056240364270560934),
 ('percent_silence', -0.67390678402142834),
 ('ring_count', 0.48483923341906693),
 ('white_proportion', 2.3131580570928114)]

### Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=15, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
list(zip(columns[1:], rf.feature_importances_))

[('length', 0.30593363755717351),
 ('last_ring_to_end', 0.33353202776482688),
 ('percent_silence', 0.15206534339705702),
 ('ring_count', 0.0086084243372190443),
 ('white_proportion', 0.19986056694372359)]

### Support Vector classifier

In [10]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', probability=True) # available kernels: linear, poly, rbf, sigmoid
svm.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Model Predictions on test set

In [11]:
## Predict on the test set
from sklearn import metrics
for m in [lr, rf, svm]:
    X_test_scaled = scaler.transform(X_test)
    y_pred = m.predict(X_test_scaled)
    # Area under the curve
    # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    print(metrics.roc_auc_score(y_test, y_pred))

0.892425845961
0.909617553453
0.913974904845


### Test each model with different metrics

In [12]:
from sklearn.pipeline import make_pipeline
# Features to use
columns = ['length', 'ring_count', 'last_ring_to_end', 'percent_silence', 'white_proportion']

X = features[columns]
y = features['label']

for m in [LogisticRegression(),RandomForestClassifier(n_estimators=20, n_jobs=-1), SVC(kernel='rbf')]:
    # First scale and then apply model
    clf = make_pipeline(preprocessing.StandardScaler(), m)
    print(m.__class__.__name__)
    
    # options for scoring: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    for scorer in ['roc_auc', 'average_precision', 'recall', 'f1']:
        scores = cross_validation.cross_val_score(clf, X, y, cv=10, scoring=scorer, n_jobs=-1)
        print("\t{}: {:.2f} (+/- {:.2f})".format(scorer, scores.mean(), scores.std() * 2))

LogisticRegression
	roc_auc: 0.96 (+/- 0.02)
	average_precision: 0.94 (+/- 0.03)
	recall: 0.90 (+/- 0.04)
	f1: 0.88 (+/- 0.03)
RandomForestClassifier
	roc_auc: 0.96 (+/- 0.02)
	average_precision: 0.95 (+/- 0.02)
	recall: 0.89 (+/- 0.04)
	f1: 0.90 (+/- 0.03)
SVC
	roc_auc: 0.96 (+/- 0.02)
	average_precision: 0.94 (+/- 0.03)
	recall: 0.91 (+/- 0.04)
	f1: 0.90 (+/- 0.02)


# Unlabeled Audios

## Load data

In [13]:
unl_features = load_features('test')
unl_features.head()

,audio_file,image_file,length,label,percent_silence,ring_count,last_ring_to_end,white_proportion
index,,,,,,,,
0,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,55.800,NaN,0.376077,2,43.251875,0.880129
1,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,65.880,NaN,0.357495,0,65.808000,0.838058
2,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,9.576,NaN,0.072220,0,9.504000,0.890358
3,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.256,NaN,0.421754,0,5.184000,0.910165
4,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.760,NaN,0.471123,0,5.688000,0.910035


## Clean data

In [14]:
# Get rid of label column
unl_features = unl_features.drop('label', axis=1)

## Check data

In [15]:
# Take a look at where nan values are
print(len(unl_features[unl_features.isnull().any(axis=1)]))
unl_features[unl_features.isnull().any(axis=1)].head()

2


,audio_file,image_file,length,percent_silence,ring_count,last_ring_to_end,white_proportion
index,,,,,,,
9746,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,29.664,0.014239,0,29.592000,NaN
12398,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,88.560,0.020017,2,49.359125,NaN


In [16]:
# just drop the remaning rows with nan values
unl_features = unl_features.dropna()

## Reset index after cleanup

In [17]:
# reset index for concatenating predicted labels
unl_features = unl_features.reset_index(drop=True)
unl_features.head()

,audio_file,image_file,length,percent_silence,ring_count,last_ring_to_end,white_proportion
0,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,55.800,0.376077,2,43.251875,0.880129
1,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,65.880,0.357495,0,65.808000,0.838058
2,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,9.576,0.072220,0,9.504000,0.890358
3,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.256,0.421754,0,5.184000,0.910165
4,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.760,0.471123,0,5.688000,0.910035


In [18]:
# Selected models 
models = [
    LogisticRegression(),
    RandomForestClassifier(n_estimators=20, n_jobs=-1),
    SVC(kernel='rbf', probability=True),
]

## Data preparation

In [19]:
# Features to use
columns = ['length', 'ring_count', 'last_ring_to_end', 'percent_silence', 'white_proportion']

y_train_all = features['label']
X_train_all = features[columns]

# Scale features
scaler = preprocessing.StandardScaler()
X_train_all_scaled = scaler.fit_transform(X_train_all)
unl_features_scaled = scaler.transform(unl_features[columns])

## Model predictions

In [20]:
# Balanced predicted labels 
labels_pred = []
# Probability distribution to minimize false positives (discarded)
proba_pred = []
# Fit model and predict for unlabeled data
for m in models:
    print('Training', m.__class__.__name__)
    m.fit(X_train_all_scaled, y_train_all)
    labels_pred.append(m.predict(unl_features_scaled))
    proba_pred.append(m.predict_proba(unl_features_scaled))

Training LogisticRegression
Training RandomForestClassifier
Training SVC


## Balanced labels analysis

In [21]:
for labels in labels_pred:
    print(np.unique(labels, return_counts=True))

(array([0, 1]), array([10313,  8657]))
(array([0, 1]), array([11031,  7939]))
(array([0, 1]), array([10793,  8177]))


### Combine predictions from models to assign the final labels

In [22]:
# Combine labels from each predictor into a matrix (one row per predictor)
agg_labels = np.vstack(labels_pred)
# Sum each column of labels
al = np.sum(agg_labels, axis=0)

# If at least two predictors predict "1", then "1", else "0"
al[np.where(al <= 1)] = 0
al[np.where(al > 1)] = 1
final_labels = al

In [23]:
np.unique(final_labels, return_counts=True)

(array([0, 1]), array([10744,  8226]))

## Unbalanced labels analysis

In [24]:
# False positive threshold
threshold = 0.9
labels_pred_unbalanced = []
for proba in proba_pred:
    labels_model = []
    for p0, p1 in proba:
        label = 1 if p1 >= threshold else 0
        labels_model.append(label)
    labels_pred_unbalanced.append(labels_model)

In [25]:
for labels in labels_pred_unbalanced:
    print(np.unique(labels, return_counts=True))

(array([0, 1]), array([15686,  3284]))
(array([0, 1]), array([13154,  5816]))
(array([0, 1]), array([14053,  4917]))


### Combine predictions from models to assign the final labels

In [26]:
# Combine labels from each predictor into a matrix (one row per predictor)
agg_labels = np.vstack(labels_pred_unbalanced)
# Sum each column of labels
al = np.sum(agg_labels, axis=0)

# If at least two predictors predict "1", then "1", else "0"
al[np.where(al <= 1)] = 0
al[np.where(al > 1)] = 1
final_labels_unb = al

In [27]:
np.unique(final_labels_unb, return_counts=True)

(array([0, 1]), array([13625,  5345]))

In [28]:
# Create a dataFrame for the predicted labels
pred_labels = pd.DataFrame(final_labels, columns=['pred_label'])
pred_labels_unb = pd.DataFrame(final_labels_unb, columns=['pred_label90'])
predicted_labels = pd.concat([pred_labels, pred_labels_unb], axis=1)
predicted_labels.head()

,pred_label,pred_label90
0,0,0
1,0,0
2,1,1
3,1,1
4,1,1


In [29]:
unlabeled_final = pd.concat([unl_features, predicted_labels], axis=1)
unlabeled_final.head()

,audio_file,image_file,length,percent_silence,ring_count,last_ring_to_end,white_proportion,pred_label,pred_label90
0,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,55.800,0.376077,2,43.251875,0.880129,0,0
1,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,65.880,0.357495,0,65.808000,0.838058,0,0
2,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,9.576,0.072220,0,9.504000,0.890358,1,1
3,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.256,0.421754,0,5.184000,0.910165,1,1
4,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,/Users/jjelosua/Developer/lanacion/ML_audio_cl...,5.760,0.471123,0,5.688000,0.910035,1,1


# Export data

In [30]:
import os
export_path = '../data/output/predicted'
if not os.path.exists(export_path):
    os.makedirs(export_path)

In [31]:
unlabeled_final.to_csv('{}/unlabeled_predicted.csv'.format(export_path), index_label='index')